In [7]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
from gensim import corpora
from gensim import models

In [8]:
# Read recipe inputs
ramen_dictionary_path = dataiku.Folder("POe5uF4H").get_path() + "/ramen_dictionary"
your_trip_advisor = dataiku.Dataset("your_trip_advisor")
df = your_trip_advisor.get_dataframe()

In [9]:
dictionary = corpora.Dictionary.load(ramen_dictionary_path)

In [10]:
dictionary.num_docs, dictionary.num_pos

(211, 251395)

In [15]:
df['jp'].values

array(['私たちは 12 時の開店時間に到着し、待ち行列で 25 分待ってから入りました。最終的に入るまでにさらに25分以上の待ち時間がありました。食べ物は本当に例外的ではありませんでした。それは...フランスの食材でできているはずです（放し飼いの鶏、私はこれが食べ物の味に交差していると思います）.私が見つけることができた唯一の肯定的な側面は、本物の経験を与えようとしている雰囲気でした.さらに表示',
       '本当の失望の総観光客の罠。食べ物は本当に塩辛く、味がしませんでした。他の 5 つ星のレビューを購入しました。このレストランは観光客を騙して生活しています。また、他の星 1 のレビューはただの嘘だと思っていましたが、食べた後はこれを思い出すでしょう。とても悲しい',
       '私が今まで食べた中で最高のラーメン。柔らかい肉、新鮮な食材、素晴らしいスープ、まともなバリエーション、私の妻のスープは驚くほど私のものとは異なり、どちらも美味しかったです.日本の市場通りにいるような、窮屈ではありますが不快ではありません。',
       '本当に素晴らしい料理と素晴らしいサービス。また、他のパリのレストランと比較して比較的安価です。パリに住んでいたとき、私のお気に入りのレストランの 1 つでした。混雑を避けたいなら閉店直前がおすすめ！',
       'バケツリストに載せる価値のあるラーメン。すべての素材の組み合わせによる味と香りの完璧さは、神体験に参加しているかのように感じさせます。インテリアは、私たちの時間と現実の認識を遊びながら明らかに追加の側面を提供します。',
       'すべてのレビュアーが話していることを試すために、約30分間並んでいます。これは素晴らしいラーメンに最適な場所だと言わざるを得ません。風味豊かなストックが特に気に入りました。レストランは日本の露店のように装飾されているので、そのレストランはとても魅力的です。',
       '開場時間の10分前に揺さぶられ、私たちの前に15〜20人ほどの列ができていたという点で、私たちはちょうどいいタイミングでした. 6:30 が過ぎると、かなり早く着席しました。彼らはあなたを非常に迅速に処理します。中は暑くて、とてもおいしいです。',
       'よくわからない場合、または何

In [0]:
# Compute recipe outputs from inputs
# TODO: Replace this part by your actual code that computes the output, as a Pandas dataframe
# NB: DSS also supports other kinds of APIs for reading and writing data. Please see doc.

trip_advisor_tf_idf_df = your_trip_advisor_df # For this sample code, simply copy input to output


# Write recipe outputs
trip_advisor_tf_idf = dataiku.Dataset("trip_advisor_tf_idf")
trip_advisor_tf_idf.write_with_schema(trip_advisor_tf_idf_df)